In [1]:
!unzip "/content/drive/MyDrive/Zyad/NN Dataset.zip" -d "/content/"

Archive:  /content/drive/MyDrive/Zyad/NN Dataset.zip
  inflating: /content/Test/0.jpg     
  inflating: /content/Test/1.jpg     
  inflating: /content/Test/10.jpg    
  inflating: /content/Test/100.jpg   
  inflating: /content/Test/101.jpg   
  inflating: /content/Test/102.jpg   
  inflating: /content/Test/103.jpg   
  inflating: /content/Test/104.jpg   
  inflating: /content/Test/105.jpg   
  inflating: /content/Test/106.jpg   
  inflating: /content/Test/107.jpg   
  inflating: /content/Test/108.jpg   
  inflating: /content/Test/109.jpg   
  inflating: /content/Test/11.jpg    
  inflating: /content/Test/110.jpg   
  inflating: /content/Test/111.jpg   
  inflating: /content/Test/112.jpg   
  inflating: /content/Test/113.jpg   
  inflating: /content/Test/114.jpg   
  inflating: /content/Test/115.jpg   
  inflating: /content/Test/116.jpg   
  inflating: /content/Test/117.jpg   
  inflating: /content/Test/118.jpg   
  inflating: /content/Test/119.jpg   
  inflating: /content/Test/12.jpg  

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation , RandomZoom , Rescaling, RandomContrast, RandomTranslation
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from matplotlib.pyplot import imshow

In [3]:
main_dir = "/content/Train/"
Labels = ['Basketball', 'Football', 'Rowing', 'Swimming', 'Tennis', 'Yoga']

#creat folders 
for i in range(len(Labels)):
  path = os.path.join(main_dir, Labels[i])
  print(path)
  os.mkdir(path) 

/content/Train/Basketball
/content/Train/Football
/content/Train/Rowing
/content/Train/Swimming
/content/Train/Tennis
/content/Train/Yoga


In [4]:
print(len(os.listdir(main_dir)))
for filename in os.listdir(main_dir):
  file_name = filename[:-4].split('_')

  image_name = main_dir+"/"+filename
  directory = "/content/Train/" +file_name[0]+"/"+filename
  print(image_name, "   ", directory)
  if len(file_name) != 1: 
    shutil.move(image_name, directory)

1687
/content/Train//Basketball_34.jpg     /content/Train/Basketball/Basketball_34.jpg
/content/Train//Football_306.jpg     /content/Train/Football/Football_306.jpg
/content/Train//Swimming_32.jpg     /content/Train/Swimming/Swimming_32.jpg
/content/Train//Yoga_183.jpg     /content/Train/Yoga/Yoga_183.jpg
/content/Train//Yoga_217.jpg     /content/Train/Yoga/Yoga_217.jpg
/content/Train//Tennis_23.jpg     /content/Train/Tennis/Tennis_23.jpg
/content/Train//Yoga_149.jpg     /content/Train/Yoga/Yoga_149.jpg
/content/Train//Rowing_39.jpg     /content/Train/Rowing/Rowing_39.jpg
/content/Train//Football_316.jpg     /content/Train/Football/Football_316.jpg
/content/Train//Basketball_13.jpg     /content/Train/Basketball/Basketball_13.jpg
/content/Train//Swimming_57.jpg     /content/Train/Swimming/Swimming_57.jpg
/content/Train//Rowing_103.jpg     /content/Train/Rowing/Rowing_103.jpg
/content/Train//Swimming_96.jpg     /content/Train/Swimming/Swimming_96.jpg
/content/Train//Swimming_97.jpg     /

In [5]:
os.mkdir("/content/Valid")

In [6]:
main_dir = "/content/Valid"

for i in range(len(Labels)):
  path = os.path.join(main_dir, Labels[i])
  print(path)
  os.mkdir(path) 

/content/Valid/Basketball
/content/Valid/Football
/content/Valid/Rowing
/content/Valid/Swimming
/content/Valid/Tennis
/content/Valid/Yoga


In [7]:
for i in range(6):
  cnt = 0
  cur_dir = "/content/Train"+"/"+Labels[i]
  num = round(len(os.listdir(cur_dir)) * 0.2)
  
  for filename in os.listdir(cur_dir):
    
    file_name = filename[:-4].split('_')
    image_name = "/content/Train"+"/"+file_name[0]+"/"+filename
    directory = "/content/Valid/" +file_name[0]+"/"+filename
    #print(image_name, "   ", directory)
    shutil.move(image_name, directory)
    cnt+=1
    if cnt == num:
      break

In [8]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [9]:
TRAIN_PATH = '/content/Train' 
VAL_PATH = '/content/Valid'


train_data_generator = image.ImageDataGenerator(
                                            rotation_range=30,
                                            zoom_range=0.15,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            )


val_data_generator = image.ImageDataGenerator()

train_generator = train_data_generator.flow_from_directory(directory= TRAIN_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE)


val_generator = val_data_generator.flow_from_directory(directory= VAL_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE)

Found 1345 images belonging to 6 classes.
Found 336 images belonging to 6 classes.


In [10]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=8,kernel_size=(3,3),padding="valid", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=16, kernel_size=(3,3), padding="valid", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding="valid", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="valid", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

In [11]:
model.add(Flatten())
model.add(Dense(units=6, activation="softmax"))

In [12]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 8)       224       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_normal.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
# Reduce Learning Rate
reduce_lr = ReduceLROnPlateau(patience=2,
                              monitor='val_loss',
                              factor=0.1)
hist = model.fit_generator(generator=train_generator, validation_data= val_generator, epochs=30,callbacks=[checkpoint,early,reduce_lr])

<ipython-input-14-3a903327ce36>:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(generator=train_generator, validation_data= val_generator, epochs=30,callbacks=[checkpoint,early,reduce_lr])


Epoch 1/30
43/43 [==============================] - ETA: 0s - loss: 7.1280 - accuracy: 0.3063

43/43 [==============================] - 29s 483ms/step - loss: 7.1280 - accuracy: 0.3063 - val_loss: 1.3761 - val_accuracy: 0.4702 - lr: 0.0010
Epoch 2/30
43/43 [==============================] - ETA: 0s - loss: 1.3594 - accuracy: 0.4684

43/43 [==============================] - 21s 488ms/step - loss: 1.3594 - accuracy: 0.4684 - val_loss: 2.0071 - val_accuracy: 0.3155 - lr: 0.0010
Epoch 3/30
43/43 [==============================] - ETA: 0s - loss: 1.4134 - accuracy: 0.4766

43/43 [==============================] - 21s 490ms/step - loss: 1.4134 - accuracy: 0.4766 - val_loss: 1.1710 - val_accuracy: 0.5268 - lr: 0.0010
Epoch 4/30
43/43 [==============================] - ETA: 0s - loss: 1.1856 - accuracy: 0.5420

...

43/43 [==============================] - 21s 470ms/step - loss: 0.5273 - accuracy: 0.8104 - val_loss: 0.5881 - val_accuracy: 0.8065 - lr: 1.0000e-08
Epoch 27/30
43/43 [==============================] - ETA: 0s - loss: 0.5587 - accuracy: 0.8000

43/43 [==============================] - 20s 476ms/step - loss: 0.5587 - accuracy: 0.8000 - val_loss: 0.5881 - val_accuracy: 0.8065 - lr: 1.0000e-08
Epoch 28/30
43/43 [==============================] - ETA: 0s - loss: 0.5560 - accuracy: 0.8030

43/43 [==============================] - 20s 472ms/step - loss: 0.5560 - accuracy: 0.8030 - val_loss: 0.5881 - val_accuracy: 0.8065 - lr: 1.0000e-09
Epoch 29/30
43/43 [==============================] - ETA: 0s - loss: 0.5123 - accuracy: 0.8156

43/43 [==============================] - 20s 473ms/step - loss: 0.5123 - accuracy: 0.8156 - val_loss: 0.5881 - val_accuracy: 0.8065 - lr: 1.0000e-09
Epoch 30/30
43/43 [==============================] - ETA: 0s - loss: 0.5667 - accuracy: 0.8037

43/43 [==============================] - 22s 516ms/step - loss: 0.5667 - accuracy: 0.8037 - val_loss: 0.5881 - val_accuracy: 0.8065 - lr: 1.0000e-10


In [15]:
model.save('VGG16.h5')

In [24]:
!cp '/content/VGG16.h5' '/content/drive/MyDrive/Zyad'

## Test

In [18]:
from tensorflow.keras.utils import img_to_array
import keras.utils as image

In [19]:
TESTING_DIR = "/content/Test"
test_images = []
for i in os.listdir(TESTING_DIR):
    img_path = TESTING_DIR + '/' + i
    #print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    test_images.append(img_array)
test_images = np.array(test_images)

In [17]:
model = load_model("/content/VGG16.h5")

In [22]:
prediction = model.predict(test_images)

22/22 [==============================] - 0s 14ms/step


In [23]:
label = ['Basketball','Football','Rowing', 'Swimming','Tennis', 'Yoga']
predicted_labels = []

for i in range(len(test_images)):
  pos = np.argmax(prediction[i], axis=0)
  print(prediction[i])
  print(pos)
  image_label = label[pos]

  if image_label == "Basketball":
    predicted_labels.append(0)
  elif image_label == "Football":
    predicted_labels.append(1)
  elif image_label == "Rowing":
    predicted_labels.append(2)
  elif image_label == "Swimming":
    predicted_labels.append(3)
  elif image_label == "Tennis":
    predicted_labels.append(4)
  elif image_label == "Yoga":
    predicted_labels.append(5)

print(predicted_labels)

[0.00646035 0.10526238 0.7524576  0.09319256 0.02689678 0.0157303 ]
2
[0.04682232 0.087634   0.16906628 0.0427014  0.6420629  0.01171314]
4
[1.16339426e-04 2.38866918e-02 1.94116086e-02 4.96840989e-03
 1.53150652e-02 9.36301947e-01]
5
[0.12665117 0.31395707 0.05843863 0.4523819  0.03962001 0.00895124]
3
[0.10896725 0.30148628 0.48814672 0.04099542 0.04346024 0.01694404]
2
[0.00376431 0.00879827 0.973584   0.00862702 0.00172519 0.00350123]
2
[0.15909456 0.29989126 0.50492406 0.0080954  0.02538147 0.00261332]
2
[1.4391177e-05 2.6909218e-05 1.7737478e-03 9.9582601e-01 2.3190018e-03
 3.9866227e-05]
3
[3.8957787e-03 9.7828650e-01 1.4223013e-03 2.9606250e-04 1.2984965e-02
 3.1143099e-03]
1
[1.3645303e-02 1.2932142e-02 9.6002400e-01 1.1905413e-03 1.1527216e-02
 6.8079040e-04]
2
[1.85131512e-07 7.99230975e-07 1.44275427e-02 9.81013477e-01
 4.44026152e-03 1.17832125e-04]
3
[0.00124966 0.00642737 0.93147236 0.01530956 0.00153633 0.0440048 ]
2
[0.00473037 0.00753169 0.00579837 0.03003883 0.019877

In [ ]:
dataframe_answer = pd.DataFrame({'image_name':  os.listdir(TESTING_DIR) , 'label': predicted_labels})

In [ ]:
dataframe_answer.to_csv('Normal.csv', index=False)